In [1]:
import numpy as np
import random as rd
from collections import deque

# PREPARATION BLOCK

In [2]:
file = open('start_swaptable','r')
start_swaptable = []
for i in file:
    start_swaptable.append(int(i)-1)
    
file = open('final_swaptable','r')
final_swaptable = []
for i in file:
    final_swaptable.append(int(i)-1)
    
file = open('f_final_swaptable','r')
f_final_swaptable = []
for i in file:
    f_final_swaptable.append(int(i)-1)

file = open('escalC','r')
escalC = []
for i in file:
    escalC.append(int(i)-1)

file = open('escalD','r')
escalD = []
for i in file:
    escalD.append(int(i)-1)
    
file = open('CP','r')
CP = []
for i in file:
    CP.append(int(i)-1)
    
S = []
for i in range(8):
    file = open('S'+str(i)+'','r')
    S.append(np.arange(64))
    k = 0
    for j in file:
        S[i][k] = int(j)
        k += 1
    S[i] = S[i].reshape((4, 16))

# DES FUNCTIONS

In [3]:
def fix_bit(number, bits = 8):
    return format(number & 2 ** bits - 1, "0{}b".format(bits))

def toBits(text):
    text_bits = "".join(fix_bit(ord(i)) for i in text)    
    if len(text_bits) % 64 != 0:
        if len(text_bits) < 64:
            text_bits = "".join(str(0) for i in range(64 - len(text_bits))) + text_bits
        elif len(text_bits) > 64:
            text_bits = "".join(str(0) for i in range(64 - len(text_bits)%64)) + text_bits
            
    return text_bits

def get_blocks_bits(text_bits):
    blocks_bits = []
    for i in range(0,len(text_bits),64):
        blocks_bits.append("".join(text_bits[i:i+64]))
    return blocks_bits

def toSigns(bits):
    return "".join(chr(int(bits[i:i+8],2)) for i in range(0,len(bits),8))

def swap_array64(ar_initial64, ar_rule):
    if len(ar_initial64) != 64:
        print("Error! len(ar_initial) != 64")
    else:
        swaped_array = [ar_initial64[i] for i in ar_rule]
        return "".join(str(i) for i in swaped_array)
    
def swap_array56(ar_initial56,ar_rule):
    if len(ar_initial56) != 56:
        print("Error! len(ar_initial) != 56")
    else:
        swaped_array = [ar_initial56[i] for i in ar_rule]
        return "".join(str(i) for i in swaped_array)

def swap_array32(ar_initial32, ar_rule):
    if len(ar_initial32) != 32:
        print("Error! len(ar_initial) != 32")
    else:
        swaped_array = [ar_initial32[i] for i in ar_rule]
        return "".join(str(i) for i in swaped_array)
    
def EP(input_quer32):
    l = 1
    output_quer48 = 48*[0]
    output_quer48[0] = input_quer32[31]
    output_quer48[1] = input_quer32[0]
    output_quer48[47] = input_quer32[0]
    output_quer48[46] = input_quer32[31]
    for i in range(2,45,5):
        output_quer48[i] = input_quer32[l]
        output_quer48[i+1] = input_quer32[l+1]
        output_quer48[i+2]= output_quer48[i+4] = input_quer32[l+2]
        output_quer48[i+3]= output_quer48[i+5] = input_quer32[l+3]
    return "".join(str(i) for i in output_quer48)

def XOR(a,b):
    if len(a) != len(b):
        print("error! len(a) != len(b)")
    else:
        c = np.array([int(i) for i in a]) ^ np.array([int(i) for i in b])
        c = list(c)
        return c
    
def f_encryp(B032,round_key48):
    arr_choice = 8*[0]
    
    output_quer48 = EP(B032)
    xor_sum = XOR(output_quer48,round_key48)
    
    block_bits = np.array(xor_sum).reshape((8,6))
    
    for i in range(8):
        arr_choice[i] = S[i][int(''.join(str(i) for i in block_bits[i][:2]),2)][int(''.join(str(i) for i in block_bits[i][2:6]),2)]
    
    pre_output_bits = "".join(fix_bit(i,4) for i in arr_choice)
    
    return swap_array32(pre_output_bits,f_final_swaptable)

def escalation_key(key64):
    array_keys = []
    C = "".join(str(i) for i in [key64[j] for j in escalC])
    D = "".join(str(i) for i in [key64[j] for j in escalD])
    for i in range(16):
        if i == 0 or i == 2 or i == 8 or i == 15:
            c = deque(list(C))
            c.rotate(-2)
            c = list(c)
            C = "".join(str(i) for i in [j for j in c])
            
            d = deque(list(D))
            d.rotate(-2)
            d = list(d)
            D = "".join(str(i) for i in [j for j in d])
            
            array_keys.append(swap_array56(C + D,CP))
        else:
            c = deque(list(C))
            c.rotate(-1)
            c = list(c)
            C = "".join(str(i) for i in [j for j in c])
            
            d = deque(list(D))
            d.rotate(-1)
            d = list(d)
            D = "".join(str(i) for i in [j for j in d])
            
            array_keys.append(swap_array56(C + D,CP))
            
    return array_keys

def enc_DES_algorithm(array64,key64):
    array_keys = escalation_key(key64)
    array64 = swap_array64(array64,start_swaptable)
    A0 = array64[0:32]
    B0 = array64[32:]
    for round_key in array_keys:
        if (round_key == array_keys[15]):
            A0 = XOR(A0, f_encryp(B0,round_key))
            break
        register = B0
        B0 = XOR(A0, f_encryp(B0,round_key))
        A0 = register
    return swap_array64(A0 + B0, final_swaptable)

def decr_DES_algorithm(array64,key64):
    array_keys = escalation_key(key64)
    array64 = swap_array64(array64,start_swaptable)
    A0 = array64[0:32]
    B0 = array64[32:]
    for i in range(15,-1,-1):
        if (i == 0):
            A0 = XOR(A0, f_encryp(B0,array_keys[i]))
            break
        register = B0
        B0 = XOR(A0, f_encryp(B0,array_keys[i]))
        A0 = register
    return swap_array64(A0 + B0, final_swaptable)

def DES_encr(text,key64):
    blocks_bits = get_blocks_bits(toBits(text))
    encr_blocks_bits = []
    for block in blocks_bits:
        encr_blocks_bits.append(enc_DES_algorithm(block,key64))
    encr_text = "".join(i for i in encr_blocks_bits)
    return toSigns(encr_text)


def DES_decr(encr_text,key64):
    encr_blocks = get_blocks_bits(toBits(encr_text))
    decr_blocks_bits = []
    for block in encr_blocks:
        decr_blocks_bits.append(decr_DES_algorithm(block,key64))
    decr_text = "".join(i for i in decr_blocks_bits)
    return toSigns(decr_text)

# TEST  BLOCK

In [4]:
text = "qwertyui"
key64 = "".join(str(rd.choice([0,1])) for i in range(64))
print(toSigns(key64))

afÈLÉË


In [12]:
text = "vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv"
encr_text = DES_encr(text,key64)
print(encr_text)
print('\n')
decr_text = DES_decr(encr_text,key64)
print(decr_text)

Åx\2ÖÈºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1ºvx\2ÖÈ1


  vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
